In [ ]:
!pip install pulp

import pandas as pd
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 6.7 MB/s 


In [1]:
import pandas as pd
from pulp import *

In [2]:
# distance_matrix = pd.read_excel('./광주광역시_경계구역_읍면동_충장동통합.xlsx', usecols = 'B : GM')
distance_matrix = pd.read_excel('./광주광역시_법정동_거리행렬.xlsx')

In [26]:
distance_matrix['광천동']['신안동']

KeyError: '신안동'

In [3]:
pop = pd.read_excel('./행정안전부_법정동_주민등록_인구수.xlsx')

In [4]:
pop_final = list(pop.iloc[:, 1]) # 인구수
final = distance_matrix[:194] # 법정동 거리행렬

In [5]:
final = final.set_index('ID')

In [6]:
final.iloc[:1, 1:] # 행,열

,광천동,덕흥동,내방동,서구양동,금호동,농성동,화정동,서구용두동,노대동,치평동,...,매곡동,북구동림동,삼거동,서석동,일곡동,연제동,덕림동,운암동,충장동,대의동
ID,,,,,,,,,,,,,,,,,,,,,
쌍촌동,2144.88004,2777.05799,1219.937464,3384.456895,2345.002069,2346.190337,1528.53187,7300.512049,7198.810977,1877.224242,...,4847.021614,3153.290374,17174.384225,6247.368919,7071.763137,5203.873351,17858.745719,3210.00283,4719.221324,5300.442001


In [7]:
region_name = final.iloc[:1,1:] # 법정동명
region_name_final = list(region_name) # 법정동명 리스트로 저장
location = region_name_final # 동 이름 전처리

In [ ]:
location

In [30]:
location = location[:5]

In [31]:
a = dict(zip(location, pop_final))

In [32]:
a

{'광천동': 48520, '덕흥동': 7805, '내방동': 451, '서구양동': 7315, '금호동': 8068}

In [33]:
## 거리행렬 정의
D = {}
for i in range(0, 193): # 딕셔너리 형태로 한 동당 거리행렬 정의
    D[i] = dict(zip(location[i:], [dict(zip(location, list(final.iloc[i,:])))]))

merged = {}
for i in range(0, 193):
    merged = {**merged, **D[i]} # 193 + 193 거리행렬 정의    

In [34]:
merged

{'광천동': {'광천동': 0.0,
  '덕흥동': 2144.88003961997,
  '내방동': 2777.05799038133,
  '서구양동': 1219.93746420492,
  '금호동': 3384.45689518103},
 '덕흥동': {'광천동': 2144.88003961991,
  '덕흥동': 0.0,
  '내방동': 3978.91567897798,
  '서구양동': 925.62967704849,
  '금호동': 2144.3059746784},
 '내방동': {'광천동': 2777.05799038136,
  '덕흥동': 3978.91567897798,
  '내방동': 0.0,
  '서구양동': 3325.46741102481,
  '금호동': 5880.42259550082},
 '서구양동': {'광천동': 1219.93746420508,
  '덕흥동': 925.629677048623,
  '내방동': 3325.46741102484,
  '서구양동': 0.0,
  '금호동': 2557.30507962959},
 '금호동': {'광천동': 3384.45689518103,
  '덕흥동': 2144.30597467835,
  '내방동': 5880.42259550079,
  '서구양동': 2557.30507962958,
  '금호동': 0.0}}

In [35]:
p = 2 # 임시선별검사소 수 정의

In [36]:
# P-Median 모델을 통해 최적지 선정하는 코드
X = LpVariable.dicts('%s_%s', (location, location), cat = 'Binary',
                     lowBound = 0,
                     upBound = 1)

In [42]:
X

{'광천동': {'광천동': 광천동_광천동,
  '덕흥동': 광천동_덕흥동,
  '내방동': 광천동_내방동,
  '서구양동': 광천동_서구양동,
  '금호동': 광천동_금호동},
 '덕흥동': {'광천동': 덕흥동_광천동,
  '덕흥동': 덕흥동_덕흥동,
  '내방동': 덕흥동_내방동,
  '서구양동': 덕흥동_서구양동,
  '금호동': 덕흥동_금호동},
 '내방동': {'광천동': 내방동_광천동,
  '덕흥동': 내방동_덕흥동,
  '내방동': 내방동_내방동,
  '서구양동': 내방동_서구양동,
  '금호동': 내방동_금호동},
 '서구양동': {'광천동': 서구양동_광천동,
  '덕흥동': 서구양동_덕흥동,
  '내방동': 서구양동_내방동,
  '서구양동': 서구양동_서구양동,
  '금호동': 서구양동_금호동},
 '금호동': {'광천동': 금호동_광천동,
  '덕흥동': 금호동_덕흥동,
  '내방동': 금호동_내방동,
  '서구양동': 금호동_서구양동,
  '금호동': 금호동_금호동}}

In [45]:
prob = LpProblem('P_Median', LpMinimize) # 목적 함수

In [46]:
# 목적 함수
prob += sum(sum(merged[i][j] * X[i][j] for j in location) for i in location)   
prob += sum(X[i][i] for i in location) == 1

In [47]:
# 제약식
for i in location: 
    prob += sum(X[i][j] for j in location) == 1

for i in location:
    for j in location: prob += X[i][j] <= X[j][j]

In [48]:
prob.writeLP("p-median_Seoul_Whole.lp")
prob.solve()

1

In [49]:
#prob

pulp.value(prob.objective)

8028.3396319078

In [66]:
X['금호동']['서구양동'].varValue

0.0

In [50]:
prob

P_Median:
MINIMIZE
3384.45689518103*광천동_금호동 + 2777.05799038133*광천동_내방동 + 2144.88003961997*광천동_덕흥동 + 1219.93746420492*광천동_서구양동 + 3384.45689518103*금호동_광천동 + 5880.42259550079*금호동_내방동 + 2144.30597467835*금호동_덕흥동 + 2557.30507962958*금호동_서구양동 + 2777.05799038136*내방동_광천동 + 5880.42259550082*내방동_금호동 + 3978.91567897798*내방동_덕흥동 + 3325.46741102481*내방동_서구양동 + 2144.88003961991*덕흥동_광천동 + 2144.3059746784*덕흥동_금호동 + 3978.91567897798*덕흥동_내방동 + 925.62967704849*덕흥동_서구양동 + 1219.93746420508*서구양동_광천동 + 2557.30507962959*서구양동_금호동 + 3325.46741102484*서구양동_내방동 + 925.629677048623*서구양동_덕흥동 + 0.0
SUBJECT TO
_C1: 광천동_광천동 + 금호동_금호동 + 내방동_내방동 + 덕흥동_덕흥동 + 서구양동_서구양동 = 1

_C2: 광천동_광천동 + 광천동_금호동 + 광천동_내방동 + 광천동_덕흥동 + 광천동_서구양동 = 1

_C3: 덕흥동_광천동 + 덕흥동_금호동 + 덕흥동_내방동 + 덕흥동_덕흥동 + 덕흥동_서구양동 = 1

_C4: 내방동_광천동 + 내방동_금호동 + 내방동_내방동 + 내방동_덕흥동 + 내방동_서구양동 = 1

_C5: 서구양동_광천동 + 서구양동_금호동 + 서구양동_내방동 + 서구양동_덕흥동 + 서구양동_서구양동 = 1

_C6: 금호동_광천동 + 금호동_금호동 + 금호동_내방동 + 금호동_덕흥동 + 금호동_서구양동 = 1

_C7: 0 광천동_광천동 <= 0

_C8: 광천동_덕흥동 - 덕흥동_덕흥동 <= 0

_C9: 광천동_내